<a href="https://colab.research.google.com/github/jonrtaylor/twitch/blob/master/example_model_kazutsugi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

train_datalink = 'https://numerai-public-datasets.s3-us-west-2.amazonaws.com/latest_numerai_training_data.csv.xz'

df_train = pd.read_csv(train_datalink)

In [ ]:
from xgboost import XGBRegressor

model = XGBRegressor(max_depth = 5, learning_rate = 0.01, n_estimators = 2000, n_jobs = -1, colsample_bytree = 0.1)#, tree_method = 'gpu_hist')

y = df_train.target_kazutsugi.values

features = [c for c in df_train if c.startswith("feature")]

X = df_train[features].values

model.fit(X, y)

In [ ]:
def correlation_score(y_true, y_pred):
    return np.corrcoef(y_true, y_pred)[0,1]
df_val = pd.read_csv('numerai_training_validation_target_nomi.csv.tar.gz')
df_val = df_val.iloc[:-1].copy()
df_val['era'] = df_val.loc[:, 'era'].str[3:].astype(int)
df_vali = df_val[df_val.era > 120].copy()
y_val = df_vali.target_nomi.values
X_val = df_vali[features].values
preds = model.predict(X_val)

In [ ]:
nomi_score = correlation_score(y_val, preds)
print(nomi_score)

In [ ]:
y_valk = df_vali.target_kazutsugi.values

kaz_score = correlation_score(y_valk, preds)
print(kaz_score)

In [ ]:
tournament_data = pd.read_csv("https://numerai-public-datasets.s3-us-west-2.amazonaws.com/latest_numerai_tournament_data.csv.xz")
X_tournament = tournament_data[features].values
predictions = model.predict(X_tournament)
predictions_df = tournament_data["id"].to_frame()
predictions_df["prediction_kazutsugi"] = predictions
predictions_df.to_csv('example_predictions_twitch.csv', index=False)

In [ ]:
from google.colab import files
files.download("example_predictions_twitch.csv")